In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf

In [2]:
##### Limit GPU for training ###
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
### read hopital names from datasets ###
df = pd.read_csv(r'hospital_v2.csv')

In [4]:
df.head()

,name
0,คลินิกกรุงเทพพัทยาสาขาจอมเทียน
1,คลินิกกรุงเทพระยองสาขาบ่อวิน
2,คลินิกกรุงเทพระยองสาขาบ้านฉาง
3,คลินิกกรุงเทพระยองสาขาปลวกแดง
4,คลินิกไทยอินเตอร์การแพทย์(เกาะพงัน)


In [21]:
from pythainlp import thai_letters

In [103]:
label_map_0 = ["0","1","2","3","4","5","6","7","8","9","-",".","/","(",")","\"","&"]
label_map_1 = list(thai_letters)
char_set = label_map_0+label_map_1

In [ ]:
char2int = { char_set[x]:x for x in range(len(char_set)) }
int2char = { char2int[x]:x for x in char_set }

In [8]:
#thresh - 0 to 1
import random
def gen_gibberish(line,thresh=0.15):
    text_true = line
    times = int(random.randrange(1,len(line)) * thresh)
    '''
    Types of replacement:
        1.Delete random character.
        2.Add random character.
        3.Replace a character.
        4.Combination?
    '''
    while times!=0:
        # try to gen noise length times...
        times-=1
        val = random.randrange(0,10)
        if val <= 5:
            #get random index
            val = random.randrange(0,10)
            index = random.randrange(2,len(line))
            if val <= 3 :
                #delete character
                line = line[:index]+line[index+1:]
            else:
                #add character
                insert_index = random.randrange(0,len(char_set))
                line = line[:index] + char_set[insert_index] + line[index:]
        else:
            index = random.randrange(0,len(char_set))
            replace_index = random.randrange(2,len(line))
            line = line[:replace_index] + char_set[index] + line[replace_index+1:]
    return line,text_true

In [5]:
hos_list = list(df['name'])
len(hos_list)

1290

In [6]:
hos_list[46]

'บ้าน'

In [106]:
hos_list2 = hos_list[:46]+hos_list[47:]

In [107]:
b = hos_list[:46]
len(b)

46

In [108]:
b2 = hos_list[47:]
len(b2)

1243

In [109]:
len(hos_list2)

1289

In [15]:
len(hos_list[:46])

46

In [16]:
if 'บ้าน' in hos_list2:
    print('vao')

In [17]:
gen_data =[]
true_data =[]
for i in range(2):
    for line in hos_list2:
        a,b = gen_gibberish(line)
        gen_data.append(a)
        true_data.append(b)

In [18]:
len(gen_data)

2578

In [19]:
a1 = ['บาน']
a2 = ['บ้า']
a3 = ['บ้านน']
a4 = ['ป้าน']
a5 =['ป้านน']
a6 =['บ็าน']
a7= ['ข้าน']
a8 = ['บานน']
a9 = ['ช้าน']
a10 = ['บ๊าน']

In [20]:
gen_data = gen_data+a1+a2+a3+a4+a5+a6+a7+a8+a9+a10

In [116]:
gen_data[12891]

'บ้า'

In [114]:
true_data = true_data +['บ้าน']*10

In [115]:
true_data[12891]

'บ้าน'

In [117]:
gen_data_comb = gen_data + hos_list
true_data_comb = true_data + hos_list

In [118]:
gen_df = pd.DataFrame(gen_data_comb)
true_df = pd.DataFrame(true_data_comb)

In [120]:
gen_df.columns = ['gen_data']
true_df.columns = ['true_data']

In [139]:
gen_df.to_csv('hos_gen.csv',index=False)
true_df.to_csv('true_gen.csv',index=False)

In [132]:
df_out = gen_df['gen_data']+true_df['true_data']

In [137]:
gen_df

,gen_data
0,คลินิกรุงเพพัทยาสาขาจคมเทียน
1,คลนิกกรุงเทพระยองสาขาบ่อวิน
2,คลินิกกร/ำงเทหระยองสาขาบา้นฉาง
3,คลินิกกรุงเทพระะธองสาขาปลวกแดง
4,คลินิกไทยอินเตอใ์การแพทย์(เกาะพงัน)
...,...
14185,อินทัชเมดิแคร์คลินิกเวชกรรมสาขาดินแดง
14186,อินทัชเมดิแคร์คลินิกเวชกรรมสาขาบางเขน
14187,อินทัชเมดิแคร์คลินิกเวชกรรมสาขาบางแค
14188,อินทัชเมดิแคร์คลินิกเวชกรรมสาขาประชาอุทิศ


In [124]:
true_df.head()

,true_data
0,คลินิกกรุงเทพพัทยาสาขาจอมเทียน
1,คลินิกกรุงเทพระยองสาขาบ่อวิน
2,คลินิกกรุงเทพระยองสาขาบา้นฉาง
3,คลินิกกรุงเทพระยองสาขาปลวกแดง
4,คลินิกไทยอินเตอร์การแพทย์(เกาะพงัน)


In [140]:
frame = gen_df.append(true_df,ignore_index=True)

In [141]:
frame.head()

,gen_data,true_data
0,คลินิกรุงเพพัทยาสาขาจคมเทียน,NaN
1,คลนิกกรุงเทพระยองสาขาบ่อวิน,NaN
2,คลินิกกร/ำงเทหระยองสาขาบา้นฉาง,NaN
3,คลินิกกรุงเทพระะธองสาขาปลวกแดง,NaN
4,คลินิกไทยอินเตอใ์การแพทย์(เกาะพงัน),NaN


In [143]:
df_col_merged =pd.concat([gen_df, true_df], axis=1)

In [144]:
df_col_merged.head()

,gen_data,true_data
0,คลินิกรุงเพพัทยาสาขาจคมเทียน,คลินิกกรุงเทพพัทยาสาขาจอมเทียน
1,คลนิกกรุงเทพระยองสาขาบ่อวิน,คลินิกกรุงเทพระยองสาขาบ่อวิน
2,คลินิกกร/ำงเทหระยองสาขาบา้นฉาง,คลินิกกรุงเทพระยองสาขาบา้นฉาง
3,คลินิกกรุงเทพระะธองสาขาปลวกแดง,คลินิกกรุงเทพระยองสาขาปลวกแดง
4,คลินิกไทยอินเตอใ์การแพทย์(เกาะพงัน),คลินิกไทยอินเตอร์การแพทย์(เกาะพงัน)


In [145]:
df_col_merged.to_csv('hospital_augment.csv',index=False)

In [7]:
def createSpace(text):
    re = ["<bow>"]
    for t in text :
        re.append(t)
        
    re.append("<eow>")
    return re

print (createSpace("sdsdsd"))

['<bow>', 's', 'd', 's', 'd', 's', 'd', '<eow>']


In [8]:
import random

In [9]:
def gen_delete(text,per=0.1):
    e = np.random.uniform() 
    if e <0.1:
        return text
    r = np.random.randint(len(list(text)))
    text_lst = list(text)
    del text_lst[r]
    return ''.join(text_lst)

In [10]:
def gen_insert(text,per=0.1):
    e = np.random.uniform() 
    if e <0.1:
        return text
    index = np.random.randint(len(list(text)))
    char = random.choice(char_set)
    text_list = list(text)
    text_list.insert(index, char)
    return ''.join(text_list)

In [11]:
def gen_substitution(text,per=0.1):
    e = np.random.uniform() 
    if e <0.1:
        return text
    index = np.random.randint(len(list(text)))
    char = random.choice(char_set)
    text_list = list(text)
    text_list[index] = char
    return ''.join(text_list)

In [110]:
delete_hos = []
delete_hos_true =[]
for i in range(12):
    for p in hos_list:
        delete_hos.append(gen_delete(p))
        delete_hos_true.append(p)

In [111]:
insert_hos = []
insert_hos_true =[]
for i in range(12):
    for p in hos_list:
        insert_hos.append(gen_insert(p))
        insert_hos_true.append(p)

In [112]:
sub_hos = []
sub_hos_true =[]
for i in range(12):
    for p in hos_list:
        sub_hos.append(gen_substitution(p))
        sub_hos_true.append(p)

In [113]:
gen_data2 = delete_hos+ insert_hos+sub_hos
true_data2 = delete_hos_true + insert_hos_true+sub_hos_true

In [114]:
len(gen_data2),len(true_data2)

(46440, 46440)

In [115]:
# using naive method 
# to remove duplicated  
# from list  
res_gen = []
res_true =[] 
for i,key in enumerate(gen_data2): 
    if key not in res_gen: 
        res_gen.append(key)
        res_true.append(true_data2[i])

In [116]:
len(res_gen),len(res_true)

(39556, 39556)

In [117]:
train_data_gen =[]
train_data_true =[]
val_data_gen =[]
val_data_true =[]
for i in range(0,len(res_gen),8):
    val_data_gen.append(res_gen[i])
    val_data_true.append(res_true[i])

In [120]:
val_data_gen[100],val_data_true[100]

('โรงพยาบาลแมสอด', 'โรงพยาบาลแม่สอด')

In [121]:
for i,key in enumerate(res_gen):
    if not key in val_data_true:
        train_data_gen.append(key)
        train_data_true.append(res_true[i])

In [122]:
len(train_data_gen),len(val_data_gen),len(train_data_true),len(val_data_true)

(76602, 4945, 76602, 4945)

In [123]:
train_data_gen[500],train_data_true[500]

('โรงพยาบลบ้านหลวง', 'โรงพยาบาลบ้านหลวง')

In [138]:
test_data_gen =[]
test_data_true =[]
for i in range(0,len(train_data_gen),12):
    test_data_gen.append(train_data_gen[i])
    test_data_true.append(train_data_true[i])

In [139]:
train_data_gen_res =[]
train_data_true_res =[]
for i,key in enumerate(train_data_gen):
    if not key in test_data_gen:
        train_data_gen_res.append(train_data_gen[i])
        train_data_true_res.append(train_data_true[i])

In [140]:
train_data_gen[0],train_data_true[0]

('คลินิกกรุงทพพัทยาสาขาจอมเทียน', 'คลินิกกรุงเทพพัทยาสาขาจอมเทียน')

In [141]:
len(train_data_true_res),len(val_data_gen),len(test_data_gen)

(63834, 4945, 6384)

In [128]:
train_gen = train_data_gen_res + hos_list
train_true = train_data_true_res +hos_list
val_gen = val_data_gen
val_true = val_data_true
test_gen = test_data_gen
test_true = test_data_true

In [129]:
gen_df_train = pd.DataFrame(train_gen)
true_df_train = pd.DataFrame(train_true)
gen_df_val =pd.DataFrame(val_gen)
true_df_val =pd.DataFrame(val_true)
gen_df_test =pd.DataFrame(test_data_gen)
true_df_test =pd.DataFrame(test_data_true)

In [130]:
gen_df_train.columns=['gen_data']
true_df_train.columns=['true_data']
gen_df_val.columns =['gen_data']
true_df_val.columns =['true_data']
gen_df_test.columns =['gen_data']
true_df_test.columns =['true_data']

In [131]:
len(gen_df_train),len(gen_df_val),len(gen_df_test)

(62570, 4945, 7661)

In [132]:
df_col_merged_train =pd.concat([gen_df_train,true_df_train],axis=1)

In [133]:
df_col_merged_val =pd.concat([gen_df_val,true_df_val],axis=1)
df_col_merged_test =pd.concat([gen_df_test,true_df_test],axis=1)

In [134]:
df_col_merged_train.to_csv('hospital_augment_train2.csv',index=False)
df_col_merged_val.to_csv('hospital_augment_val2.csv',index=False)
df_col_merged_test.to_csv('hospital_augment_test2.csv',index=False)

In [87]:
df_col_merged2.to_csv('hospital_augment2_val.csv',index=False)

In [167]:
len(sub_hos)

12900

In [109]:
df_test = pd.read_csv('hospital_augment_test.csv')

In [110]:
df_test_gen = list(df_test['gen_data'])
df_test_true = list(df

In [142]:
df_test_read = pd.read_csv('hospital_augment_test2.csv')

In [148]:
list_test_gen = list(df_test_read['gen_data'])
list_test_true = list(df_test_read['true_data'])

In [149]:
test_gen_res =[]
test_true_res=[]
for i in range(0,len(list_test_gen),2):
    test_gen_res.append(list_test_gen[i])
    test_true_res.append(list_test_true[i])

In [150]:
gen_df_test2=pd.DataFrame(test_gen_res)
true_df_test2 =pd.DataFrame(test_true_res)

In [151]:
len(gen_df_test2)

3831

In [152]:
gen_df_test2.columns =['gen_data']
true_df_test2.columns =['true_data']

In [153]:
df_col_merged_test =pd.concat([gen_df_test2,true_df_test2],axis=1)

In [154]:
df_col_merged_test.to_csv('hospital_augment_test2.csv',index=False)